# Step 5 Classifier with Semantic Kernel ResponsesAgent
### This script results in an error due to SK’s image handling.

In [1]:
# !pip install "semantic-kernel>=1.27.0" --quiet


In [2]:
import os, glob, asyncio
import base64
from pathlib import Path
from dotenv import load_dotenv
from typing import List, Literal
from pydantic import BaseModel, Field
from semantic_kernel.agents import AzureResponsesAgent
from semantic_kernel.connectors.ai.open_ai import AzureOpenAISettings
from semantic_kernel.contents import (
    AuthorRole, ChatMessageContent, TextContent, ImageContent
)
load_dotenv(override=True)


True

In [3]:
class Step5Response(BaseModel):
    confidence: int = Field(
        description="Classification Score (1 means NOT the 5_hexwrench_tight step, 5 means it IS the 5_hexwrench_tight step)"
    )
    reasoning: str

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        base64_encoded = base64.b64encode(image_file.read()).decode("utf-8")
    return f"data:image/jpeg;base64,{base64_encoded}"


In [4]:
system_prompt_path = Path("./system_message_11.txt")
with system_prompt_path.open("r", encoding="utf-8") as f:
    system_prompt = f.read()

items = [
    TextContent(text=system_prompt)
]

In [5]:
# Folder containing sample images (change if needed)
sample_folder = "./output/5_hexwrenchtight/"

#items = [
#    TextContent(text="The following images are examples of the 5_hexwrenchtight step in the assembling process of a chair.")
#]
items.append(TextContent(text="The following images are examples of the 5_hexwrenchtight step in the assembling process of a chair."))

# Load every *.jpg in folder
image_files = sorted(glob.glob(os.path.join(sample_folder, "*.jpg")))
for idx, img_path in enumerate(image_files, start=1):
    items.append(TextContent(text=f"sample{idx} – {os.path.basename(img_path)}"))
    items.append(ImageContent(encode_image(img_path)))

items.append(TextContent(text="The following three images are consecutive frames from a video clip. Based on these images, classify whether they correspond to step 5_hexwrench_tight in the chair assembly process."))

# 4. Test images
test_folder = "./output/"
test_images = [
    "frame_0063_t63.0s.jpg",
    "frame_0064_t64.0s.jpg",
    "frame_0065_t65.0s.jpg",
]

for i, img_name in enumerate(test_images, start=1):
    img_path = os.path.join(test_folder, img_name)
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Test image {img_path} not found.")
    items.append(TextContent(text=f"test{i}"))
    items.append(ImageContent(encode_image(img_path)))

user_msg = ChatMessageContent(role=AuthorRole.USER, items=items)
print(f"Total images attached: {sum(isinstance(it, ImageContent) for it in items)}")

Total images attached: 10


In [6]:
'''# Folder containing sample images (change if needed)
sample_folder = "./output/5_hexwrenchtight/"

#items = [
#    TextContent(text="The following images are examples of the 5_hexwrenchtight step in the assembling process of a chair.")
#]
items.append(TextContent(text="The following images are examples of the 5_hexwrenchtight step in the assembling process of a chair."))

# Load every *.jpg in folder
image_files = sorted(glob.glob(os.path.join(sample_folder, "*.jpg")))
for idx, img_path in enumerate(image_files, start=1):
    items.append(TextContent(text=f"sample{idx} – {os.path.basename(img_path)}"))
    items.append(ImageContent.from_image_file(img_path))

items.append(TextContent(text="The following three images are consecutive frames from a video clip. Based on these images, classify whether they correspond to step 5_hexwrench_tight in the chair assembly process."))

# 4. Test images
test_folder = "./output/"
test_images = [
    "frame_0063_t63.0s.jpg",
    "frame_0064_t64.0s.jpg",
    "frame_0065_t65.0s.jpg",
]

for i, img_name in enumerate(test_images, start=1):
    img_path = os.path.join(test_folder, img_name)
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Test image {img_path} not found.")
    items.append(TextContent(text=f"test{i}"))
    items.append(ImageContent.from_image_file(img_path))

user_msg = ChatMessageContent(role=AuthorRole.USER, items=items)
print(f"Total images attached: {sum(isinstance(it, ImageContent) for it in items)}")'''

'# Folder containing sample images (change if needed)\nsample_folder = "./output/5_hexwrenchtight/"\n\n#items = [\n#    TextContent(text="The following images are examples of the 5_hexwrenchtight step in the assembling process of a chair.")\n#]\nitems.append(TextContent(text="The following images are examples of the 5_hexwrenchtight step in the assembling process of a chair."))\n\n# Load every *.jpg in folder\nimage_files = sorted(glob.glob(os.path.join(sample_folder, "*.jpg")))\nfor idx, img_path in enumerate(image_files, start=1):\n    items.append(TextContent(text=f"sample{idx} – {os.path.basename(img_path)}"))\n    items.append(ImageContent.from_image_file(img_path))\n\nitems.append(TextContent(text="The following three images are consecutive frames from a video clip. Based on these images, classify whether they correspond to step 5_hexwrench_tight in the chair assembly process."))\n\n# 4. Test images\ntest_folder = "./output/"\ntest_images = [\n    "frame_0063_t63.0s.jpg",\n    "f

In [7]:
client = AzureResponsesAgent.create_client()

agent = AzureResponsesAgent(
    ai_model_id=AzureOpenAISettings().responses_deployment_name,   # <- Responses deployment
    client=client,
    instructions="You are an expert in analysing furniture‑assembly images. "
                 "Answer in JSON only, following the schema.",   
    name="Step5Classifier",
)


In [8]:
async def classify():
    response = await agent.get_response(messages=[user_msg])
    print("Raw output:\n", response.content)
    try:
        result = Step5Response.model_validate_json(response.content)
        print("\nParsed:", result)
    except Exception as e:
        print("\n⚠️ Parse error:", e)

await classify()



AgentExecutionException: ("<class 'semantic_kernel.agents.open_ai.azure_responses_agent.AzureResponsesAgent'> failed to complete the request", BadRequestError("Error code: 400 - {'error': {'message': 'Too many images in request. Max is 50.', 'type': None, 'param': None, 'code': 'BadRequest'}}"))